<a href="https://colab.research.google.com/github/MasaYan24/lecture_2021_phys_gakushuin/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST demo

荒い一桁の手書き数字を 0-9 に分類するタスクです。

MNIST: Modified National Institute of Standards and Technology databas

ライブラリを読み込む設定 (必ず最初に実行)

In [ ]:
from typing import List

import matplotlib.pyplot as plt
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from torch import nn, optim

データの読み込みと確認

In [ ]:
digits_data = datasets.load_digits()
print(f"data type={type(digits_data)}")  # データタイプ表示
print(f"elements={dir(digits_data)}")  # 構成要素表示 ['DESCR', 'data', 'images', 'target', 'target_names']
print(f"DESCR={digits_data.DESCR}")  # データの内容についての概要
print(f"data.shape={digits_data.data.shape}")  # data の形式表示 (1797, 64): 8x8=64 のサイズのデータが 1797 個
print(f"data[0]={digits_data.data[0].reshape(8,8)}")  # 最初のデータ ゼロの位置に数値がある。
print(f"images.shape={digits_data.images.shape}")  # images の形式表示 (1797, 8, 8): 8x8 のサイズのデータが 1797 個
print(f"images[0]={digits_data.images[0]}")  # 最初のデータ ゼロの位置に数値がある。data と images の違いは形が 1D か 2D か
print(f"target.shape={digits_data.target.shape}")  # target の形状
print(f"target[0-50]={[i for i in digits_data.target[0:50]]}")  # target は画像がそれぞれ何に対応しているかを示している。ランダムな並びのようだ。
print(f"target_names={digits_data.target_names}")  # データセットに含まれる種類。0-9 の 10 種類。

# 最初の 10 個を画像で表示
n_img = 10
plt.figure(figsize=(10, 4))
for i in range(n_img):
   ax = plt.subplot(2, 5, i + 1)
   plt.imshow(digits_data.images[i], cmap="Greys_r")
   # plt.imshow(digits_data.data[i].reshape(8, 8), cmap="Greys_r")  # 上と同じこと
   ax.get_xaxis().set_visible(False)
   ax.get_yaxis().set_visible(False)
plt.show()

学習用と検証用にデータ分割を行う。

In [ ]:
digit_1D_images = digits_data.data
labels = digits_data.target
x_train, x_test, t_train, t_test = train_test_split(digit_1D_images, labels, random_state=0)  # random_state を変更すると分割が変わる

x_train = torch.tensor(x_train, dtype=torch.float32)  # 学習データ
t_train = torch.tensor(t_train, dtype=torch.int64)  # 学習データの答え t: target
x_test = torch.tensor(x_test, dtype=torch.float32)  # 検証データ
t_test = torch.tensor(t_test, dtype=torch.int64)  # 検証データの答え
print(f"x_train.shape, t_train.shape, x_test.shape, t_test.shape={x_train.shape}, {t_train.shape}, {x_test.shape}, {t_test.shape}")

ニューラルネットワーク (net)、ロス関数 (loss_func)、Optimizer (optimizer) の設定。

In [ ]:
# ネットワークの形: 64 (入力) → 32 → 16 → 10 (出力) の前結合ネットワーク
net = nn.Sequential(
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 10),
)
print(net)

loss_fnc = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.01)  # 確率的勾配効果法

学習実行

In [ ]:
record_loss_train: List[float] = []  # 逐次ロスの格納場所
record_loss_test: List[float] = []

epochs = 1000
for i in range(epochs):
    optimizer.zero_grad()  # 勾配の初期化

    y_train = net(x_train)  # 現状のモデルで学習データをを推論
    y_test = net(x_test)  # 現状のモデルで検証データを推論

    loss_train = loss_fnc(y_train, t_train)  # ロス関数の計算
    loss_test = loss_fnc(y_test, t_test)
    record_loss_train.append(loss_train.item())  # 現状のロスを格納
    record_loss_test.append(loss_test.item())

    loss_train.backward()  # 誤差逆伝播を実行

    optimizer.step()  # 誤差逆伝播の結果を使って weight と bias を更新

    if i % 100 == 0:  # 100 ステップごとに途中結果を出力
        print(f"Epoch: {i}, Loss_Train: {loss_train}, Loss_Test: {loss_test}")

結果をグラフで表示

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(range(len(record_loss_train)), record_loss_train, label="Train")
ax.plot(range(len(record_loss_test)), record_loss_test, label="Test")
ax.legend()

ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
plt.show()

正解率を計算

In [ ]:
y_test = net(x_test)
count = (y_test.argmax(1) == t_test).sum().item()
print(f"correct rate: {count/len(y_test)*100:.2f}%")

テスト画像について選んで推論を実行

In [ ]:
img_ids = [0, 1, 2]  # 何番目のイメージを使うかここで指定

fig, axes = plt.subplots(1, len(img_ids))
axes = axes if len(img_ids) != 1 else [axes]
for ith, ax in enumerate(axes):
    ax.imshow(x_test[img_ids[ith]].reshape(8, 8), cmap="Greys_r")
    pred = net(x_test[img_ids[ith]]).argmax().item()
    ax.set_title(f"pred:{pred}, ans:{t_test[img_ids[ith]]}")